# Stokes equation

In this tutorial we present how to solve a three-fields Stokes equation with [PyGeoN](https://github.com/compgeo-mox/pygeon). The unkwons are the vorticity $r$, the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\mu$ the viscosity, we want to solve the following problem: find $(r, q, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\mu^{-1} r - \nabla \times q = 0 \\
\nabla \times r + \nabla p = {0} \\
\nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ \left\{
\begin{array}{ll}
\begin{array}{l} 
\nu \times r = 1 \\
\nu \cdot q = {0}
\end{array}
& \text{on } \partial_{top} \Omega
\\
\begin{array}{l} 
p = 0 \\
\nu \times q = {0}
\end{array}
& \text{on } \partial \Omega \setminus \partial_{top} \Omega
\end{array}
\right.
$$
Since the pressure is defined up to a constant, we fix its value in a cell. Other choices can be made.<br>

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we will use linear Lagrange elements for $r$ and Raviart-Thomas for ${q}$ we are restricted to triangles. In this example we consider a bi-dimensional grid, but the presented code will work also in 3d where $r$ is approximated with Nédélec elements. Since for $r$ we need ridges, we need to convert the PorePy grid to a PyGeoN grid and then do the `compute_geometry` to compute the informations for the ridges.

In [29]:
# make the mixed-dimensional grid by using pp functionalities
domain = {"xmin": 0, "xmax": 1, "ymin": 0, "ymax": 1}
network = pp.create_fracture_network(domain=domain)

mesh_size = 1/20
mesh_kwargs = {"mesh_size_frac": mesh_size, "mesh_size_min": mesh_size}

mdg = network.mesh(mesh_kwargs)
# convert the mixed-dimensional grid into a pg mixed-dimensional grid
pg.convert_from_pp(mdg)
mdg.compute_geometry()

With the following code we set the data, in particular the viscosity and the boundary conditions. Since we need to identify each side of $\partial \Omega$ for $r$ and $q$ we need few steps.

In [30]:
# set the data
keyword = "flow"
bc_val, bc_ess, source = [], [], []
for sd, data in mdg.subdomains(return_data=True):

    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    b_faces = sd.tags["domain_boundary_faces"]
    ridge_c = sd.nodes

    ess_faces = b_faces

    ess_ridges = np.zeros(sd.num_ridges, dtype=bool)
    top_ridges = np.logical_and.reduce((ridge_c[1, :] == 1, ridge_c[0, :] != 0, ridge_c[0, :] != 1))

    ess_cells = np.zeros(sd.num_cells, dtype=bool)
    ess_cells[0] = True

    bc_faces = np.zeros(sd.num_faces)
    bc_ridges = np.zeros(sd.num_ridges)
    bc_ridges[top_ridges] = 1

    bc_val.append(np.hstack((bc_ridges, bc_faces, np.zeros(sd.num_cells))))
    bc_ess.append(np.hstack((ess_ridges, ess_faces, ess_cells)))

    source.append(np.zeros(sd.num_faces))

    parameters = {
        "second_order_tensor": pp.SecondOrderTensor(np.ones(sd.num_cells)),
    }
    data[pp.PARAMETERS] = {keyword: parameters}
    data[pp.DISCRETIZATION_MATRICES] = {keyword: {}}

source = np.hstack(source)
bc_val = np.hstack(bc_val)
bc_ess = np.hstack(bc_ess)

Once the data are assigned to the mixed-dimensional grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
M & -B^\top_r & \\
B_r & & -B^\top_q\\
& B_q &
\end{array}
\right)
\left(
\begin{array}{c} 
r\\
q\\ 
p
\end{array}
\right)
=\left(
\begin{array}{c} 
q_{\partial}\\
p_{\partial}\\ 
0
\end{array}
\right)
$$<br>
where $q_{\partial}$ and $p_{\partial}$ is the vector associated to the flux and pressure boundary contions on the natural boundary of $\partial \Omega$. To construct the saddle-point problem, we rely on the `scipy.sparse` function `bmat`. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [31]:
# construct the local matrices
mass = pg.ridge_mass(mdg)
face_mass = pg.face_mass(mdg)
curl = face_mass * pg.curl(mdg)
div = pg.div(mdg)

# assemble the saddle point problem
spp = sps.bmat([[mass, -curl.T,   None], 
                [curl,    None, -div.T], 
                [None,     div,   None]], format="csc")

# get the degrees of freedom for each variable
_, dof_r = curl.shape
dof_p, dof_q = div.shape

# assemble the right-hand side
rhs = bc_val
rhs[dof_r:dof_r+dof_q] += face_mass * source

We solve the linear system and extract the three solutions $r$, $q$ and $p$.

In [33]:
# solve the problem
ls = pg.LinearSystem(spp, rhs)
ls.flag_ess_bc(bc_ess, np.zeros(bc_ess.size))
x = ls.solve()

# extract the variables
r = x[:dof_r]
q = x[dof_r:dof_r+dof_q]
p = x[-dof_p:]

Since the computed $r$ and $q$ are one value per ridge and facet, respectively, of the grid, for visualization purposes we project the vorticity and velocity in each cell. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [34]:
# post process vorticity
proj_r = pg.eval_at_cell_centers(mdg, pg.Lagrange(keyword))
cell_r = proj_r * r

# post process velocity
proj_q = pg.proj_faces_to_cells(mdg)
cell_q = (proj_q * q).reshape((3, -1), order="F")

# save the solutions to be exported in the data dictionary of the mdg
for _, data in mdg.subdomains(return_data=True):
    data[pp.STATE] = {"cell_r": cell_r, "cell_q": cell_q, "p": p}

# export the solutions
save = pp.Exporter(mdg, "sol")
save.write_vtu(["cell_r", "cell_q", "p"])

A representation of the computed solution is given below, where the cells are colored with $r$ and the arrows are the $q$. <br>
![](fig/stokes.png)